# Protein deep learning

Although proteins are more of an interest for bioinformaticians than for cheminformaticians, they are a crucial element of drug design. Especially the 3D structure of proteins is very important because of its connection to the functioning of these biological machines in the organism.

## Protein representations

Proteins are chemical compounds, but considering their huge volume, they are often processed in a different way than small molecules. Typical representations are:

1. **Amino acid sequence** - proteins are strings of amino acids that can be represented as text, and by adding information about dihedral angles we can encode spatial information; we can process this representation using recurrent neural networks or transformers;
![dihedrals](https://upload.wikimedia.org/wikipedia/commons/c/c0/Protein_backbone_PhiPsiOmega_drawing.jpg)
2. **Graphs** - this representation is similar to molecular graphs known from the previous notebooks, but in this case to simplify the structure, only α carbons are represented encoding their amino acid residue; this representation can be processed using graph neural networks;
3. **Voxel grids** - this representation is constructed by surrounding the protein with a voxel grid (cubes that encode atoms they intersect); usually the voxel size can be set to 1 Å^3 (1 Å = 10^-10 m); this representation can be processed using 3D convolutional neural networks;
![voxel](https://journals.plos.org/plosone/article/file?id=10.1371/journal.pone.0221347.g002&type=large)
4. **Distance or contact maps** - two-dimensional representation that represents distances between alpha carbons; we can binarize these distances using a selected threshold value, and such maps are called contact maps; this representation can be processed using (2D) convolutional neural networks;
![contact](https://upload.wikimedia.org/wikipedia/commons/8/8d/Protein_Contact_Map%2C_2-Color%2C_2QIP-A.png)
5. **Point clouds** - sets of not connected points (alpha carbons) encoding selected features (e.g. amino acid codes); this representation can be processed by neural networks adapted to processing point clouds, which are similar to graph neural networks;
6. **Meshes** - in this representation we only encode features of the protein surface; a graph is constructed by sampling a grid of points on the surface of proteins (a mesh), and the nodes can encode features related to the amino acids in the vicinity; this representation can be processed by graph neural networks.

## Problems that can be solved

1. **Inverse protein folding**, also known as protein design - a task that aims to find an amino acid sequence that folds to a given 3D structure;
2. **Binding site detection** - discovering cavities in proteins that can be used as binding sites for ligands; it can be realized as 3D grid segmentation using 3D convolutions (e.g. U-Net);
3. **Binding affinity prediction** - evaluation of the binding affinity given the ligand-protein complex (e.g. a ligand docked to the crystal structure of a protein).
4. **protein-based *de novo* design** - designing new ligands that match the binding site of a given protein.
![deepsite](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSv_unCC0dhDSQWwQipcKQSEulT064aq-WOnQ&usqp=CAU)

**Exercise 1:** Implement one of the representations above (the code below can be used to implement 3D voxel grids).

**Zadanie 2:** Implement a network that processes this representation (you don't need to train it, it should only return a correct output). In the case of voxel grids: check if your voxel grid can be passed through a 3D convolution (imported from PyTorch), and the output shape is what you expected.

## Przykładowy fragment implementacji wokseli

In [5]:
from rdkit import Chem
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook

In [6]:
symbols = ['C', 'N', 'O', 'S']


In [7]:
# Load protein from a PDB file
mol = Chem.MolFromPDBFile('../L4-molecular-docking/4ivt.pdb')

In [8]:
pos = mol.GetConformer().GetPositions()
pos.shape

(2911, 3)

In [9]:
bounding_box  = np.floor(pos.min(axis=0)), np.ceil(pos.max(axis=0))
bounding_box

(array([-10., -11., -12.]), array([55., 49., 35.]))

In [10]:
box_size = (bounding_box[1] - bounding_box[0]).astype(int)
box_size

array([65, 60, 47])

In [11]:
grid = np.zeros([*box_size, 4])

In [12]:
for i, atom in enumerate(mol.GetAtoms()):
    symbol = atom.GetSymbol()
    position = np.floor(pos[i] - bounding_box[0]).astype(int)
    grid[position[0], position[1], position[2], symbols.index(symbol)] = 1.


    # Map atoms onto the grid, use 1 to mark the corresponding grid position and channel
    # tip: you will need the atom symbol and atom position aligned with the grid

In [13]:
grid.max(), grid.min()

(1.0, 0.0)

In [34]:
# The code below will plot your voxel grid
fig = plt.figure(figsize=(7, 5))
ax = fig.add_subplot(111, projection='3d')
ax.voxels(grid[:, :, :, 0], facecolors='grey', alpha=0.7)
ax.voxels(grid[:, :, :, 1], facecolors='blue', alpha=0.7)
ax.voxels(grid[:, :, :, 2], facecolors='red', alpha=0.7)
ax.voxels(grid[:, :, :, 3], facecolors='orange', alpha=0.7)
plt.show()

In [14]:
grid.shape

(65, 60, 47, 4)

In [15]:
#deepsite

In [28]:
import torch
model = torch.nn.Sequential(
    torch.nn.Conv3d(in_channels=4, out_channels=32, kernel_size=8, padding=2),
    torch.nn.ELU(),
    torch.nn.Conv3d(in_channels=32, out_channels=48, kernel_size=4, padding=1),
    torch.nn.ELU(),
    torch.nn.MaxPool3d(kernel_size=2),
    torch.nn.Dropout3d(0.25),
    torch.nn.Flatten(),
    torch.nn.Linear(in_features=10368, out_features=256),
    torch.nn.ELU(),
    torch.nn.Linear(in_features=256, out_features=1),
    torch.nn.Sigmoid()
)

In [29]:
x = torch.FloatTensor(grid[:16, :16 ,:16, :]).unsqueeze(0).movedim(4,1)

In [30]:
model(x)

tensor([[0.5130]], grad_fn=<SigmoidBackward0>)